In [1]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the SpaceX launch data into a pandas DataFrame
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Determine the maximum and minimum payload values
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a list of dropdown options for launch sites
dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}]
dropdown_options.extend([{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()])

# Create a Dash application
app = dash.Dash(__name__)

# Define the layout of the Dash application
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # Dropdown menu to select launch site
    html.Br(), 
    html.Label("Select Launch Site"),
    dcc.Dropdown(id='site-dropdown',
                 options=dropdown_options,
                 value='ALL',
                 placeholder='Select a Launch Site',
                 searchable=True),
    html.Br(),
    
    # Pie chart to show the total successful launches count for all sites or a specific site
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # Slider to select payload range
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(id='payload-slider',
                    min=min_payload,
                    max=max_payload,
                    step=1000,
                    value=[min_payload, max_payload],
                    marks={str(payload): str(payload) for payload in [min_payload, max_payload]}),

    # Scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# Callback for updating the pie chart based on selected launch site
# TASK 2: Callback for pie chart
@app.callback(Output('success-pie-chart', 'figure'),
              [Input('site-dropdown', 'value')])
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Calculate the proportion of successful launches (class=1) for all launch sites
        success_proportions = spacex_df.groupby('Launch Site')['class'].mean()
        fig = px.pie(names=success_proportions.index, values=success_proportions.values, title='Proportion of Successful Launches for All Sites')
    else:
        # Filter the DataFrame to include only data for the selected site
        df_site = spacex_df[spacex_df['Launch Site'] == selected_site]
        # Calculate the proportion of successful launches (class=1) for the selected site
        success_proportion = df_site['class'].mean()
        fig = px.pie(names=[f'{selected_site} (Success)', f'{selected_site} (Failed)'], 
                     values=[success_proportion, 1 - success_proportion], 
                     title=f'Proportion of Successful Launches for {selected_site}')
    return fig


# Callback for updating the scatter chart based on selected launch site and payload range
@app.callback(Output('success-payload-scatter-chart', 'figure'),
              [Input('site-dropdown', 'value'),
               Input('payload-slider', 'value')])
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    if selected_site == 'ALL':
        filtered_df = spacex_df
        title = 'Payload vs. Success Rate for All Launch Sites'
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        title = f'Payload vs. Success Rate for {selected_site}'

    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= low) & (filtered_df['Payload Mass (kg)'] <= high)]
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title=title)
    return fig

# Run the Dash application
if __name__ == '__main__':
    app.run_server()



C:\Users\braya\AppData\Local\Temp\ipykernel_16368\3696916627.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\braya\AppData\Local\Temp\ipykernel_16368\3696916627.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
